In [1]:
! pip install -r requirements.txt

  Using cached sentence_transformers-2.5.1-py3-none-any.whl (156 kB)
  Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
  Using cached notebook-7.2.2-py3-none-any.whl (5.0 MB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl (24 kB)
  Using cached jupyterlab-4.2.5-py3-none-any.whl (11.6 MB)
  Using cached nbconvert-7.16.4-py3-none-any.whl (257 kB)
  Using cached jupyter_server-2.14.2-py3-none-any.whl (383 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl (69 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl (13 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl (59 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
  Using cached nbclient-0.10.0-py3-none-any.whl (25 kB)
  Using cached jupyter_events-0.10.0-py3-none-any.whl (18 kB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\santi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Rag From Scratch v1

## What RAG stands for?

Rag stands for Retrieval agumented generation

The goal of RAG is to take information an pass it to a LLM so it can generate an output using said information

* Retrieval: Search relevant information given a query, e.g "what are the most useful Data Quality Checks"

* Augmented: Take the relevant information we get from retrieval and augment our input (prompt) to an LLM with that relevant information

* Generation: Take the two first steps and pass them to an LLM for generative outputs.

## Why RAG?

The main goal of RAG are to improve the output of LLMs

1. Prevent hallucinations
2. Use Custom relevant data, Generates specific responses using specific documents instead of general answers.


In [2]:
!nvidia-smi

Tue Sep  3 14:05:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4080      WDDM  | 00000000:05:00.0  On |                  N/A |
|  0%   50C    P0              44W / 340W |    918MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Lets work on the pdf

First we load it and apply some text cleaning. formatting is important since the clearer the text, the clearer the LLM will undertand.
While saving the pdf content in a list, we attach some metadata with it, to be able to do some exploratory analysis on it, using pandas for example.

In [3]:
! pip install PyMuPDF tqdm


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import os
import requests
import fitz # requires: !pip install PyMuPDF
from tqdm.auto import tqdm # pip isntall tqdm
# PDF path

pdf_path = "../sources/OReillys Data Quality Fundamentals.pdf"

def text_formatter(text: str) -> str:
    """Perform minor formatting on text."""
    cleaned_text = text.replace("\n", "").strip()
    
    return cleaned_text

def open_and_read_pdf(pdf_path: str)-> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text=text_formatter(text=text)
        pages_and_texts.append({
            "page_number": page_number - 41,
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count_raw": len(text.split(". ")),
            "page_token_count": len(text) / 4, # 1 token = 4 characters
            "text": text
        })
    return pages_and_texts

C:\Users\santi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pages_and_text=open_and_read_pdf(pdf_path=pdf_path)
pages_and_text[:2]

310it [00:00, 1064.49it/s]


[{'page_number': -41,
  'page_char_count': 141,
  'page_word_count': 20,
  'page_sentence_count_raw': 1,
  'page_token_count': 35.25,
  'text': "Barr Moses,  Lior Gavish &  Molly VorwerckData Quality FundamentalsA Practitioner's Guide to Building  Trustworthy Data PipelinesSponsored by"},
 {'page_number': -40,
  'page_char_count': 2445,
  'page_word_count': 366,
  'page_sentence_count_raw': 9,
  'page_token_count': 611.25,
  'text': 'DATA“A must-read for anyone who cares about data quality.”—Debashis SahaData Leader  AppZen, Intuit, and eBayData Quality FundamentalsUS $59.99  CAN $74.99ISBN: 978-1-098-11204-2Twitter: @oreillymedialinkedin.com/company/oreilly-mediayoutube.com/oreillymedia Do your product dashboards look funky? Are your quarterly reports stale? Is the data set you’re using broken or just plain wrong? These problems affect almost every team, yet they’re usually addressed on an ad hoc basis and in a reactive manner. If you answered yes to these questions, this book is for 

In [6]:
import random
random.sample(pages_and_text, k=3)

[{'page_number': 140,
  'page_char_count': 961,
  'page_word_count': 149,
  'page_sentence_count_raw': 5,
  'page_token_count': 240.25,
  'text': 'If you sit on a centralized data team (Figure 6-16), fielding and routing these incidentalerts to the appropriate owners requires a bit more foresight and planning. A cen‐tralized data team reports directly to the CDO or head of data and simultaneouslyfields queries and incidents from data corresponding to different business units.Unless otherwise specified, several business units are its stakeholders. There are prosand cons to both data team structures when it comes to ensuring data reliability, butmore on this in Chapter 8.Figure 6-16. Centralized data team structureEither way, we suggest you set up dedicated Slack channels for data pipelines ownedand maintained by specific members of your data team, inviting relevant stakeholdersso they’re in the know if critical data they rely on is down. Many teams we work withset up PagerDuty or Opsgen

In [7]:
import pandas as pd

p_df= pd.DataFrame(pages_and_text)
p_df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,310.00,310.00,310.00,310.00,310.00
mean,113.50,2025.95,338.53,8.89,506.49
std,89.63,763.01,324.07,12.39,190.75
min,-41.00,0.00,1.00,1.00,0.00
25%,36.25,1490.25,223.25,5.00,372.56
50%,113.50,2244.50,321.00,7.00,561.12
75%,190.75,2591.50,386.00,10.00,647.88
max,268.00,4408.00,3098.00,134.00,1102.00


## Why would we care about page token count?

Token count is important to thin about because:
1. Embedding models don't deal with infinite tokens, For example an embedding model may have been trainer to embed sequences of 384 tokens into numerical space (sentence-transformers `all-mpnet-base-v2`)

2. LLMs don't deal with infinite tokens, they can't accept infinite tokens in their context window.

#### So it will be wise to instead of using the limit of tokens with all the characters from our PDF, we should just use useful characters so we get good tokens: Good data in, good data out


## Further text processing (splitting pages into sentences)
Two ways to do that:
1. split sentences using split(". ")
2. We can do this with a NLP library such as spaCy and nltk

In [16]:
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_sm

  Using cached pip-24.2-py3-none-any.whl (1.8 MB)
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     --- ------------------------------------ 0.1/1.3 MB 3.0 MB/s eta 0:00:01
     -------------- ------------------------- 0.5/1.3 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 10.0 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-airflow-providers-presto 5.5.2 requires pandas<2.2,>=2.1.2; python_version >= "3.9", but you have pandas 2.2.1 which is incompatible.
apache-airflow-providers-apache-hive 8.1.2 requires pandas<2.2,>=2.1.2; python_version >= "3.9", but you have pandas 2.2.1 which is incompatible.
apache-airflow-providers-apache-hdfs 4.5.0 requires pandas<2.2,>=2.1.2; python_version >= "3.9", but you have pandas 2.2.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\santi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "C:\Users\santi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\spacy\__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "C:\Users\santi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Py

In [15]:
from spacy.lang.en import English

nlp = English()

# Add sentencizer
nlp.add_pipe("sentencizer")

# Create a document instance as an example

doc=nlp("This is a sentence. This another sentence. I like elephants.")
assert len(list(doc.sents)) == 3

# Print out our sentences split
list(doc.sents)

OSError: [WinError 126] No se puede encontrar el módulo especificado. Error loading "C:\Users\santi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.